### Importing the required libraries

In [ ]:
#For right now pandas for the df's is all I need to import
import pandas as pd

### Importing the data

In [ ]:
#For this proof of concept, I'm simply going to use the 2019-2020 data
df = pd.read_csv("data/shots_2019.csv")
#I will need the list of team codes for later
team_lst = df['teamCode'].unique().tolist()

### Filter down to 5v5 even strength

In [ ]:
def get_stats(dataframe,team_name):
    filter_1 = dataframe['awaySkatersOnIce']==5
    filter_2 = dataframe['homeSkatersOnIce']==5
    filter_3 = dataframe['awayEmptyNet']==0.0
    filter_4 = dataframe['homeEmptyNet']==0.0
    filter_5 = dataframe['isPlayoffGame']==0.0
    df_filtered = dataframe[filter_1 & filter_2 & filter_3 & filter_4 & filter_5]
    
    #I will also filter out the un used columns at this point
    new_col_lst = ['awayTeamCode', 'game_id', 'homeTeamCode', 'isHomeTeam', 'team', 'teamCode', 'xGoal']
    df_filtered = df_filtered.filter(new_col_lst, axis=1)
    
    #Next to grab the median value for xGoals for this scenario
    #even_mean = df_filtered['xGoal'].mean()
    even_mean = df_filtered['xGoal'].median()
    
    #I will have a total of 6 filters
    #The first two check to make sure the team we are checking for is involved in that stat line
    is_home = df_filtered['homeTeamCode'] == team_name#team_lst[team_num]
    is_away = df_filtered['awayTeamCode'] == team_name#team_lst[team_num]

    #The next two check to see if the event was by or against the team in question
    event_for = df_filtered['teamCode'] == team_name#team_lst[team_num]
    event_against = df_filtered['teamCode'] != team_name#team_lst[team_num]

    #The last two will check if the xGoal figure is equal to or above the median or below it 
    above_mean = df_filtered['xGoal'] >= even_mean
    below_mean = df_filtered['xGoal'] < even_mean
    
    aboveFor = len(df_filtered[(is_home | is_away) & event_for & above_mean])
    belowFor = len(df_filtered[(is_home | is_away) & event_for & below_mean])
    aboveAgainst = len(df_filtered[(is_home | is_away) & event_against & above_mean])
    belowAgainst = len(df_filtered[(is_home | is_away) & event_against & below_mean])
    
    #now to append this data to a stat list
    stat_lst = []
    stat_lst.append(team_name)
    stat_lst.append(aboveFor)
    stat_lst.append(belowFor)
    stat_lst.append(aboveAgainst)
    stat_lst.append(belowAgainst)
    stat_lst.append(round((aboveFor/(aboveFor+aboveAgainst)*100),2))
    stat_lst.append(round((belowFor/(belowFor+belowAgainst)*100),2))
    stat_lst.append(round((aboveFor/(aboveFor+belowFor)*100),2))
    stat_lst.append(round((aboveAgainst/(aboveAgainst+belowAgainst)*100),2))
    
    return stat_lst

### Make a list of lists with each team

In [ ]:
league_lst = []
for team in team_lst:
    league_lst.append(get_stats(df,team))

### Turn into a dataframe

In [ ]:
headers = ['Team','aboveFor','belowFor','aboveAgainst','belowAgainst',
           'abovePercent','belowPercent','forPercent','againstPercent']
df_final = pd.DataFrame(league_lst, columns = headers)

In [ ]:
df_final.head(31)